In [23]:
import io
import os
import os.path
import matplotlib.pyplot as plt
from PIL import Image
from io import BytesIO
from heic2png import HEIC2PNG
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from googleapiclient.http import MediaIoBaseDownload

SCOPES = ['https://www.googleapis.com/auth/drive.file']
MIME_TYPE = 'application/vnd.google-apps.document'
APPLICATION_NAME = 'ipa-google-drive-api-client'

def get_service():

    # credentialの取得
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'google-drive-api.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
        
    # serviceの取得
    service = build('drive', 'v3', credentials=creds) 
    
    return service

def read_ocr(service, input_file, lang='en'):

    # ファイルのアップロード

    # ローカルファイルの定義
    media_body = MediaFileUpload(input_file, mimetype=MIME_TYPE, resumable=True)

    # Google Drive上のファイル名
    newfile = 'output.pdf'

    body = {
        'name': newfile,
        'mimeType': MIME_TYPE
    }

    # 　creat関数でファイルアップロード実行
    # 同時にOCR読み取りも行う
    output = service.files().create(
        body=body,
        media_body=media_body,
        # ここで読み込み先言語の指定を行う
        ocrLanguage=lang,
    ).execute()

    # テキストファイルのダウンロード

    # リクエストオブジェクト生成
    request = service.files().export_media(
        fileId=output['id'],
        mimeType = "text/plain"
    )

    #  出力用テキストファイル名
    output_path = 'output.txt'

    fh = io.FileIO(output_path, "wb")
    downloader = MediaIoBaseDownload(fh, request)
    done = False
    while done is False:
        status, done = downloader.next_chunk()

    # Google Drive上のファイル削除
    service.files().delete(fileId=output['id']).execute()
 
    # テキストの取得
    with open(output_path) as f:
        mylist = f.read().splitlines()[1:]

    return mylist

service = get_service()

In [24]:
#heic2png(要は拡張子変更)
if __name__ == '__main__':
    png_imgfile = HEIC2PNG('input/IMG_3988.HEIC')
    png_imgfile.save() # it'll save as (samename).png

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Hoshi Kairi\\test\\input\\IMG_3988.HEIC'

In [28]:
png_imgfile = 'input/IMG_3988_edit.jpg'

In [29]:
# コンフィグ
COMPRESS_QUALITY = 90 # 圧縮のクオリティ⇒15MB低度なら調節不要だけど随時、、、
# ファイル名を取得
file_name = os.path.splitext(os.path.basename(png_imgfile))[0]
with open(png_imgfile, 'rb') as inputfile:
    # バイナリモードファイルをPILイメージで取得
    im = Image.open(inputfile)
    # JPEG形式に変換して、圧縮を実行
    im = im.convert('RGB')
    im_io = BytesIO()
    im.save(im_io, 'JPEG', quality = COMPRESS_QUALITY)
with open('comp_' + file_name + '.jpg', mode='wb') as outputfile:
    # 出力ファイル(comp_png_image.png)に書き込み
    outputfile.write(im_io.getvalue())

In [30]:
# 日本語の場合

input_file = outputfile.name ##bufferwriter型からパスを取得
output = read_ocr(service, input_file, 'ja')

UnicodeDecodeError: 'cp932' codec can't decode byte 0xef in position 0: illegal multibyte sequence